## **Welcome to the LabTA Scheduler**

In [1]:
from schedule import Schedule
import input_creator
import stats
import graph
import solver
import helpers
import pandas as pd

## **Default Schedule Inputs**

These inputs are the defaults based on the Fall 2019 LabTA Schedule. Please edit the title of the csv file to being your own csv file. Change the slotdict keys and values as needed for changes in the slots and number of shifts per slot in the schedule.

In [2]:
#Default csv file
csv_file = 'default_input.csv'

#dict of target number of students in each slot
slotdict = {"Mo_1900" : 8, "Mo_2100" : 7,"Tu_1900" : 5, "Tu_2100" : 4,"We_1900" : 4, "We_2100" : 4,"Th_1900" : 4, "Th_2100" : 4,"Fr_1900" : 4, "Fr_2100" : 4,"Sa_1500" : 5, "Sa_1600" : 6,"Sa_1700" : 5,"Su_1700" : 4,"Su_1800" : 3,"Su_1900" : 6,"Su_2000" : 4, "Su_2100" : 6}


In [3]:
duration = 120 #length of slots (in minutes)

#default column values
gap = 180 #length of break students need between shifts (in minutes)
cap = 2 #number of shifts a TA can/wants to work
exp = 3 #everyone is 'experienced' (exp > 0) by default
skill = 4 #everyone is 'skilled' (skill > 2) by default

#list of slots that need more skilled TA's
stress_slots = []

#numeric value indicating how many TAs the scheduler can hire above the targeted value for any given slot
target_delta = 1

#number of shifts the scheduler can assign in addition to the slotdict shift numbers
flex_shifts = 3

#sets minimum number of experienced TA's per slot
MIN_EXP = 1

#sets minimum number of skilled TA's per stress slot
MIN_SKILL = 2

#gets number of slots
num_slots = 0
for slot in slotdict:
    num_slots += slotdict[slot]
    
#Default weights
weight_dict = {}
weight_dict['slot_type'] = 8
weight_dict['no_1'] = 6
weight_dict['guarantee_shift'] = 6
weight_dict['avail'] = 5
weight_dict['shift_cap'] = 6
weight_dict['equality'] = 2

In [4]:
def results(weight_dict, input_csv, slotdict):
    
    df = input_creator.get_df(csv_file)
    students = list(df['name'])
    make_display = input_creator.check_col(df, gap, cap, exp, skill)
    exp_dict = helpers.get_dict(df, 'experience')
    skill_dict = helpers.get_dict(df, 'skill')

    #dict of slots to check as keys, and overlapping slots as values (student won't be placed in overlap)
    slots = input_creator.get_slots(df)

    #dict of slots and their prev slots
    prev_slot = input_creator.get_prev_slots(df, duration)

    #create graph nodes and weight edges
    graph_data = graph.create_graph(df, weight_dict, slotdict, prev_slot, num_slots, duration)
    student_nodes = graph_data[0]
    slot_nodes = graph_data[1]
    wt = graph_data[2]

    #solve the problem, get the ordered schedule, updated df
    results = solver.run_solver(student_nodes, slot_nodes, wt, df, slotdict, MIN_EXP, MIN_SKILL, stress_slots, target_delta, flex_shifts, duration)
    sched = results[0]
    df = results[1]

    #get stats
    happiness_stats = stats.hap_stats(df, sched)
    student_stats = stats.stud_stats(df, sched, prev_slot)
    corr_stats = stats.corr_stats(df, sched)
    slot_stats = stats.slotsize_stats(sched, slotdict)
    
    #color and display the dataframe
    df['wrong type'] = student_stats[3]
    cols = ['name']
    cols2 = list(slotdict.keys())
    cols.extend(cols2)
    cols3 = ['happiness', 'wrong type', 'slot_type', 'availability', 'cap', 'experience', 'skill', 'hours']
    cols.extend(cols3)
    df = df[cols]
    styled_df = df.style.\
        applymap(helpers.color_schedule, subset = pd.IndexSlice[:, sched.schedule.keys()]).\
        apply(helpers.color_wrong_type, subset=['wrong type'])
    display(styled_df)
    
    #print out stats
    print("Max Weight schedule stats:")
    if make_display[0]:
        stats.exp_stats(exp_dict, sched)
    if make_display[1]:
        stats.skill_stats(skill_dict, sched)
    
    #Evaluate happiness stats of schedule
    print('Average Happiness: ', happiness_stats[0])
    print()
    print('Standard deviation of happiness: ', happiness_stats[1])
    print()
    print('Min happy outlier students: ', happiness_stats[2])
    print()
    print('Availability to happiness correlation: ', corr_stats[0])
    print()
    print('Skill to happiness correlation: ', corr_stats[1])
    print()
    print('Experience to happiness correlation: ', corr_stats[2])
    print()
    print('Students who got 1\'s: ', student_stats[0])
    print()
    print('Students who got wrong slot type: ', student_stats[1])
    print()
    print('Students without shifts: ', student_stats[2])
    print()
    print('Understaffed slots: ', slot_stats[0])
    print()
    print('Flex Shifts created: ', slot_stats[1])
    print()
    
    return(sched)
    

In [5]:
max_weight_sched = results(weight_dict, csv_file, slotdict)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pulp/pulp.py:1114: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Status: Optimal


,name,Mo_1900,Mo_2100,Tu_1900,Tu_2100,We_1900,We_2100,Th_1900,Th_2100,Fr_1900,Fr_2100,Sa_1500,Sa_1600,Sa_1700,Su_1700,Su_1800,Su_1900,Su_2000,Su_2100,happiness,wrong type,slot_type,availability,cap,experience,skill,hours
0,Ben Schiffer,-3,-3,1,1,0,0,2,2,0,0,1,1,1,3,3,0,3,3,100,False,4,27,2,3,4,4
1,Urvashi Uberoy,-3,1,3,1,-3,2,3,2,0,0,0,0,0,1,1,0,0,0,100,False,2,20,2,3,4,4
2,Tajreen Ahmed,-3,1,3,1,-3,1,3,1,0,0,0,0,0,0,0,0,0,0,100,False,2,16,2,3,4,4
3,Aniela Macek,3,3,0,0,0,1,2,2,1,1,0,0,0,0,0,-3,3,-3,100,False,0,22,2,3,4,4
4,Kyle Johnson,-3,1,1,1,3,1,1,1,2,1,0,0,0,-2,2,1,0,0,83,False,2,20,2,3,4,4
5,Somya Arora,1,1,0,1,-3,3,0,3,3,-3,2,0,0,0,0,1,0,0,100,False,2,21,2,3,4,4
6,Khyati Agrawal,-3,1,2,1,3,2,-3,2,1,0,1,1,1,1,1,1,1,1,100,False,2,26,2,3,4,4
7,Donovan Coronado,0,0,0,0,1,1,1,1,3,1,1,-3,1,-2,2,1,0,0,83,False,0,18,2,3,4,4
8,Annie Zhou,-3,1,0,1,3,2,0,0,-3,2,2,0,0,1,0,1,1,1,100,False,2,21,2,3,4,4
9,Sandun Bambarandage,-3,3,0,0,2,2,0,0,2,2,1,1,2,3,-3,1,1,1,100,False,0,27,2,3,4,4


Max Weight schedule stats:
Average Happiness:  94.81481481481484

Standard deviation of happiness:  8.573966809256977

Min happy outlier students:  ['Catherine Yu']

Availability to happiness correlation:  0.202114261031682

Skill to happiness correlation:  nan

Experience to happiness correlation:  nan

Students who got 1's:  []

Students who got wrong slot type:  []

Students without shifts:  []

Understaffed slots:  {}

Flex Shifts created:  {'Mo_1900': 1, 'We_1900': 1, 'Th_1900': 1}



## **Print Schedule:**

In [6]:
max_weight_sched.print_sched()

Mo_1900 : 
['Annie Zhou', 'Ariel Rakovitsky', 'Ben Schiffer', 'Khyati Agrawal', 'Kyle Johnson', 'Nala Sharadjaya', 'Sandun Bambarandage', 'Tajreen Ahmed', 'Urvashi Uberoy']
Mo_2100 : 
['Ben Schiffer', 'Caroline di Vittorio', 'Cathleen Kong', 'HJ Suh', 'Nala Sharadjaya', 'Yashodhar Govil', 'Ze Xin Koh']
Tu_1900 : 
['Akash Pattnaik', 'Alan Ding', 'Bobby Morck', 'Caroline di Vittorio', 'Sahan Paliskara']
Tu_2100 : 
['Bobby Morck', 'Justin Chang', 'Sahan Paliskara', 'Shaurya Garg']
We_1900 : 
['Arjun Devraj', 'Michelle Woo', 'Somya Arora', 'Tajreen Ahmed', 'Urvashi Uberoy']
We_2100 : 
['Alan Ding', 'Arjun Devraj', 'Charlie Smith', 'Michelle Woo']
Th_1900 : 
['Avi Bendory', 'Caio Costa', 'Charlie Smith', 'Kawin Tiyawattanaroj', 'Khyati Agrawal']
Th_2100 : 
['Avi Bendory', 'Jason Xu', 'Kawin Tiyawattanaroj', 'Niranjan Shankar']
Fr_1900 : 
['Annie Zhou', 'Connie Miao', 'Lily Zhang', 'Nathan Alam']
Fr_2100 : 
['Lily Zhang', 'Nathan Alam', 'Ryan Golant', 'Somya Arora']
Sa_1500 : 
['Aditya Kohli

## **Advanced Schedule Inputs**

These inputs are for customizing the schedule based on more detailed preferences

In [7]:
#custom weights
weight_dict_custom = {}
weight_dict_custom['slot_type'] = 5
weight_dict_custom['no_1'] = 7
weight_dict_custom['guarantee_shift'] = 6
weight_dict_custom['avail'] = 6
weight_dict_custom['shift_cap'] = 3
weight_dict_custom['equality'] = 3

In [8]:
#list of slots that need more skilled TA's
stress_slots = ["Mo_1900", "Mo_2100", "Su_2000", "Su_2100"]

#sets minimum number of skilled TA's per stress slot
MIN_SKILL = 2

#numeric value indicating how many TAs the scheduler can hire above the targeted value for any given slot
target_delta = 2

#number of shifts the scheduler can assign in addition to the slotdict shift numbers
flex_shifts = 4

#sets minimum number of experienced TA's per slot
MIN_EXP = 3


In [9]:
custom_schedule = results(weight_dict_custom, csv_file, slotdict)

/Users/aaronlichtblau/opt/miniconda3/envs/testsched/lib/python3.8/site-packages/pulp/pulp.py:1195: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Status: Optimal
Aditya Kohli - Mo_1900
Akash Pattnaik - Mo_1900
Annie Zhou - Mo_1900
Ariel Rakovitsky - Mo_1900
Khyati Agrawal - Mo_1900
Kyle Johnson - Mo_1900
Tajreen Ahmed - Mo_1900
Urvashi Uberoy - Mo_1900
Ben Schiffer - Mo_2100
Caroline di Vittorio - Mo_2100
Cathleen Kong - Mo_2100
HJ Suh - Mo_2100
Sandun Bambarandage - Mo_2100
Yashodhar Govil - Mo_2100
Ze Xin Koh - Mo_2100
Bobby Morck - Tu_1900
Caroline di Vittorio - Tu_1900
Sahan Paliskara - Tu_1900
Uri Schwartz - Tu_1900
Urvashi Uberoy - Tu_1900
Alan Ding - Tu_2100
Bobby Morck - Tu_2100
Sahan Paliskara - Tu_2100
Shaurya Garg - Tu_2100
Arjun Devraj - We_1900
Charlie Smith - We_1900
Jason Xu - We_1900
Khyati Agrawal - We_1900
Kyle Johnson - We_1900
Michelle Woo - We_1900
Alan Ding - We_2100
Arjun Devraj - We_2100
Charlie Smith - We_2100
Justin Chang - We_2100
Avi Bendory - Th_1900
Catherine Yu - Th_1900
Kawin Tiyawattanaroj - Th_1900
Niranjan Shankar - Th_1900
Tajreen Ahmed - Th_1900
Caio Costa - Th_2100
Kawin Tiyawattanaroj - Th_

,name,Mo_1900,Mo_2100,Tu_1900,Tu_2100,We_1900,We_2100,Th_1900,Th_2100,Fr_1900,Fr_2100,Sa_1500,Sa_1600,Sa_1700,Su_1700,Su_1800,Su_1900,Su_2000,Su_2100,happiness,wrong type,slot_type,availability,cap,experience,skill,hours
0,Ben Schiffer,3,3,1,1,0,0,2,2,0,0,1,1,1,3,3,0,3,3,100,True,4,27,2,3,4,4
1,Urvashi Uberoy,3,1,3,1,3,2,3,2,0,0,0,0,0,1,1,0,0,0,100,False,2,20,2,3,4,4
2,Tajreen Ahmed,3,1,3,1,3,1,3,1,0,0,0,0,0,0,0,0,0,0,100,False,2,16,2,3,4,4
3,Aniela Macek,3,3,0,0,0,1,2,2,1,1,0,0,0,0,0,3,3,3,100,False,0,22,2,3,4,4
4,Kyle Johnson,3,1,1,1,3,1,1,1,2,1,0,0,0,2,2,1,0,0,100,False,2,20,2,3,4,4
5,Somya Arora,1,1,0,1,3,3,0,3,3,3,2,0,0,0,0,1,0,0,100,True,2,21,2,3,4,4
6,Khyati Agrawal,3,1,2,1,3,2,3,2,1,0,1,1,1,1,1,1,1,1,100,False,2,26,2,3,4,4
7,Donovan Coronado,0,0,0,0,1,1,1,1,3,1,1,3,1,2,2,1,0,0,100,False,0,18,2,3,4,4
8,Annie Zhou,3,1,0,1,3,2,0,0,3,2,2,0,0,1,0,1,1,1,100,False,2,21,2,3,4,4
9,Sandun Bambarandage,3,3,0,0,2,2,0,0,2,2,1,1,2,3,3,1,1,1,100,False,0,27,2,3,4,4


Max Weight schedule stats:
Average Happiness:  97.40740740740742

Standard deviation of happiness:  6.108814995639827

Min happy outlier students:  []

Availability to happiness correlation:  0.10291884434225228

Skill to happiness correlation:  nan

Experience to happiness correlation:  nan

Students who got 1's:  []

Students who got wrong slot type:  ['Ben Schiffer', 'Somya Arora', 'Nathan Alam', 'Avi Bendory', 'Lily Zhang', 'Michelle Woo', 'Nala Sharadjaya', 'Catherine Yu']

Students without shifts:  []

Understaffed slots:  {}

Flex Shifts created:  {'We_1900': 2, 'Th_1900': 1}



In [10]:
custom_schedule.print_sched()

Mo_1900 : 
['Aditya Kohli', 'Akash Pattnaik', 'Annie Zhou', 'Ariel Rakovitsky', 'Khyati Agrawal', 'Kyle Johnson', 'Tajreen Ahmed', 'Urvashi Uberoy']
Mo_2100 : 
['Ben Schiffer', 'Caroline di Vittorio', 'Cathleen Kong', 'HJ Suh', 'Sandun Bambarandage', 'Yashodhar Govil', 'Ze Xin Koh']
Tu_1900 : 
['Bobby Morck', 'Caroline di Vittorio', 'Sahan Paliskara', 'Uri Schwartz', 'Urvashi Uberoy']
Tu_2100 : 
['Alan Ding', 'Bobby Morck', 'Sahan Paliskara', 'Shaurya Garg']
We_1900 : 
['Arjun Devraj', 'Charlie Smith', 'Jason Xu', 'Khyati Agrawal', 'Kyle Johnson', 'Michelle Woo']
We_2100 : 
['Alan Ding', 'Arjun Devraj', 'Charlie Smith', 'Justin Chang']
Th_1900 : 
['Avi Bendory', 'Catherine Yu', 'Kawin Tiyawattanaroj', 'Niranjan Shankar', 'Tajreen Ahmed']
Th_2100 : 
['Caio Costa', 'Kawin Tiyawattanaroj', 'Nala Sharadjaya', 'Niranjan Shankar']
Fr_1900 : 
['Annie Zhou', 'Connie Miao', 'Donovan Coronado', 'Somya Arora']
Fr_2100 : 
['Lily Zhang', 'Nathan Alam', 'Ryan Golant', 'Somya Arora']
Sa_1500 : 
['Anu